#### New QAT frontdoor getting started (EXPERIMENTAL)

This is a demo of the new QAT 'frontdoor' which uses the experimental QAT pipelines.

We aren't using this in production yet and your mileage may vary.

In [ ]:
%load_ext autoreload
%autoreload 2

Import QAT and instantiate a qat instance

In [ ]:
from qat import QAT

qat = QAT()

Default pipelines are setup by default...

In [ ]:
qat.pipelines

Define a program...

In [ ]:
src = """
OPENQASM 3;
bit[2] c;
qubit[2] q;
h q;
measure q -> c;
"""

Compile it with the default pipeline...

In [ ]:
pkg, metrics = qat.compile(src)

Execute the compiled program with the default pipeline...

In [ ]:
res, metrics = qat.execute(pkg)
res

Comile and execute a program with a different pipeline...

In [ ]:
pkg, metrics = qat.compile(src, pipeline="echo16")
res, metrics = qat.execute(pkg, pipeline="echo16")

QAT pipelines can also be configured with YAML...

In [ ]:
qat = QAT(qatconfig="../qatconfig.eg.yaml")
qat.pipelines

The yaml looks like this... (It's not very flexible yet it will be)

In [ ]:
import pathlib
import yaml


print(yaml.dump(yaml.safe_load(pathlib.Path("../qatconfig.eg.yaml").read_text())))

Change the default pipeline

In [ ]:
qat.pipelines.set_default("echo8-alt")
inst, metrics = qat.compile(src)
res, metrics = qat.execute(inst)
res

Run a program as a one liner...

In [ ]:
res, metrics = qat.run(src)
res

Make a custom pipeline

In [ ]:
from qat import Pipeline

In [ ]:
from qat.backend.waveform_v1.codegen import WaveformV1Backend
from qat.engines.waveform_v1 import EchoEngine
from qat.frontend import AutoFrontend
from qat.middleend.middleends import DefaultMiddleend
from qat.purr.backends.echo import get_default_echo_hardware
from qat.runtime.runtimes.simple import SimpleRuntime

model = get_default_echo_hardware(qubit_count=16)
P = Pipeline(
    name="mycoolnewpipeline",
    frontend=AutoFrontend(model),
    middleend=DefaultMiddleend(model),
    backend=WaveformV1Backend(model),
    runtime=SimpleRuntime(engine=EchoEngine()),
    model=model,
)

Compile and execute against the new pipeline

In [ ]:
pkg, metrics = qat.compile(src, pipeline=P)
res, metrics = qat.execute(pkg, pipeline=P)
res

Keep it around for later...

In [ ]:
qat.pipelines.add(P)
P.name

Now it's available by name

In [ ]:
pkg, metrics = qat.compile(src, pipeline="mycoolnewpipeline")
res, metrics = qat.execute(pkg, pipeline="mycoolnewpipeline")
res